In [1]:
import ee
ee.Initialize()

In [2]:
import geepyGLAD as glad

In [3]:
import ipygee as ui

In [4]:
provincias = ee.FeatureCollection('users/rprincipe/norte_argentina')

In [5]:
def convert_provincias(feat):    
    n = ee.String(feat.get('NAM'))

    n = ee.String(ee.Algorithms.If(n.compareTo('Córdoba'), n, 'Cordoba'))
    n = ee.String(ee.Algorithms.If(n.compareTo('San Juan'), n, 'San_Juan'))
    n = ee.String(ee.Algorithms.If(n.compareTo('Santa Fe'), n, 'Santa_Fe'))
    n = ee.String(ee.Algorithms.If(n.compareTo('Santiago del Estero'), n, 'Santiago_del_Estero'))
    n = ee.String(ee.Algorithms.If(n.compareTo('Tucumán'), n, 'Tucuman'))

    return feat.set('NAME', ee.String(n))

In [6]:
provincias = provincias.map(convert_provincias)

In [7]:
import ee
from geepyGLAD import alerts, utils
import requests
import os
import math
from time import time

def downloadFile(url, name, ext, path=None):
    """ Download a file from a given url

    :param url: full url
    :type url: str
    :param name: name for the file (can contain a path)
    :type name: str
    :param ext: extension for the file
    :type ext: str
    :return: the created file (closed)
    :rtype: file
    """
    response = requests.get(url, stream=True)
    code = response.status_code

    if path is None:
        path = os.getcwd()

    path = os.path.join(path, name)

    while code != 200:
        if code == 400:
            return None
        response = requests.get(url, stream=True)
        code = response.status_code
        size = response.headers.get('content-length', 0)
        if size: print('size:', size)

    filename = '{}.{}'.format(path, ext)

    with open(filename, "wb") as handle:
        for data in response.iter_content():
            handle.write(data)

    return handle


class Timer:
    def __init__(self):
        self.start = time()
        
    def ping(self, name):
        dt = time() - self.start
        print('{} {}s'.format(name, dt))

        
def _download(image, region, name, limit=100, total_download=300,
              extension='JSON', path=None, overwrite=False):
    
    vector = utils.make_vector(image, region)
        
    iterations = math.ceil(total_download/limit)

    for i in range(iterations):
        i += 1
        vlist = vector.toList(limit, limit*i)
        
        v = ee.FeatureCollection(vlist)
        
        name_i = '{}_{}'.format(name, i)
        
        exists = os.path.exists(name_i)
        if exists and overwrite or not exists:        
            url = v.getDownloadURL(**{
                    'filetype': extension,
                    'filename': name_i
                  })
            print('Downloading {}...'.format(name_i))
            exts = {
                'JSON': 'geojson',
                'KML': 'kml'
            }
            downloadFile(url, name_i, exts[extension], path)
        else:
            print('{} already exists'.format(path))


def download_probable(site, date, limit=1, smooth='max', property_name=None,
                      path=None, extension='JSON', features_per_file=100,
                      total_features=300):
    """ Download probable alert vector. Parameter `site` can be a
    FeatureCollection in which case will be splitted with `property_name`
    parameter
    """
    if isinstance(site, (ee.FeatureCollection, ee.Feature)):
        geom = site.geometry()
    else:
        geom = site
    
    ui.eprint(geom)
    
    alert = alerts.get_probable(geom, date, limit, smooth)
    
    if path is None:
        path = os.path.join(os.getcwd(), 'alerts')
    
    if isinstance(site, ee.FeatureCollection) and property_name:
        names = utils.get_options(site, property_name)
        names_cli = names.getInfo()
        for name in names_cli:
            region = site.filterMetadata(
                property_name, 'equals', name).first().geometry()

            count = utils.histogram(alert, 'probable', region)
            count_cli = count.getInfo()
            
            if count_cli > 0:
                filename = 'probable_alert_for_{}_{}'.format(date, name)
                try:
                    _download(alert, region, filename, features_per_file,
                              total_features, extension, path)
                except Exception as e:
                    print(str(e))
                    continue
            else:
                print('No probable alerts for day {} in {}'.format(date, name))
    else:
        count = utils.histogram(alert, 'probable', geom)
        count_cli = count.getInfo()
        
        if count_cli > 0:
            filename = 'probable_alert_for_{}'.format(date)
            _download(alert, geom, filename, features_per_file,
                      total_features, extension, path)
        else:
            print('No probable alerts for day {} in the given region'.format(date))

In [28]:
download_probable(provincias, '2019-04-20', property_name='NAME')

No probable alerts for day 2019-04-20 in Jujuy
No probable alerts for day 2019-04-20 in Salta
vector 0.002544403076171875s
vlist 0.003669261932373047s
v 0.003816366195678711s
Computation timed out.
No probable alerts for day 2019-04-20 in Chaco
No probable alerts for day 2019-04-20 in Santiago_del_Estero
vector 0.002554178237915039s
vlist 0.002859830856323242s
v 0.002984762191772461s
url 222.97966384887695s
vlist 224.61663269996643s
v 224.61685705184937s
url 229.57265973091125s
vlist 231.1037757396698s
v 231.10389232635498s
url 236.20857977867126s
vector 0.0020825862884521484s
vlist 0.002346038818359375s
v 0.0024476051330566406s
Computation timed out.
No probable alerts for day 2019-04-20 in La Rioja
No probable alerts for day 2019-04-20 in San_Juan
No probable alerts for day 2019-04-20 in Mendoza
No probable alerts for day 2019-04-20 in San Luis
No probable alerts for day 2019-04-20 in Cordoba
No probable alerts for day 2019-04-20 in Santa_Fe
vector 0.002082347869873047s
vlist 0.00236

KeyboardInterrupt: 

In [8]:
tuc = provincias.filterMetadata('NAME', 'equals', 'Tucuman').first()
# ui.eprint(tuc)
download_probable(tuc, '2019-04-20')

EEException: Feature, argument 'geometry': Invalid type. Expected: Geometry. Actual: Feature.